### Database with 4*4 images

shapes : square, triangle, cirle
colors : light red, light blue, light yellow, light green

dataset size : 1,000,000

probability to have a symbol at each position : 0.5

Same as v0.0.1 but paths are relative to the db root and not absolute



In [2]:
db_root_path = "/home/jleguy/Documents/postdoc/data/XAIGenImg/db0.1.2/"

In [3]:
from dbimg import load_db, generate_uuid

db = load_db(db_root_path)

In [3]:
# Number of images generated
NBGEN = 1000000

# Grid division of each image
X_DIVISIONS = 4
Y_DIVISIONS = 4

# Size of the images in pixels
img_size = (700, 700)

# Probability to generate a geometrical shape at each position in the grid
SHAPE_PROB = 0.5

# Define available shapes
SHAPES = ['circle', 'square', 'triangle']
COLORS  = ["#F86C62", "#7AB0CD", "#F4D67B", "#87C09C"]

In [4]:
import numpy as np
from dbimg import generate_uuid
import os
import tqdm

for genidx in tqdm.tqdm(range(NBGEN)):
    content = []
    for i in range(X_DIVISIONS):
        for j in range(Y_DIVISIONS):
            if np.random.random() < SHAPE_PROB:
                content.append({
                    "shape": np.random.choice(SHAPES),
                    "pos": (i, j),
                    "color": np.random.choice(COLORS)
                })

    imgid = generate_uuid()
    db[imgid] = {
        "path": os.path.join("img", imgid + ".png"),
        "division" : (X_DIVISIONS, Y_DIVISIONS),
        "size": img_size,
        "content": content
    }

100%|██████████| 1000000/1000000 [01:41<00:00, 9814.20it/s]


In [5]:
from genimg import gen_img_and_save_db
gen_img_and_save_db(db, db_root_path, overwrite=False, n_jobs=36)

100%|██████████| 1000000/1000000 [17:24<00:00, 957.47it/s]


### Create dataset with blue diagonal rule

In [4]:
def blue_diagonal_rule(img_content):
    symbols_in_diagonal = 0
    for c in img_content:
        if c["pos"][0] == c["pos"][1]:
            symbols_in_diagonal += 1
            if c["color"] != "#7AB0CD":
                return False
    return symbols_in_diagonal >= 1

In [5]:
from gendataset import create_dataset_based_on_rule
import os
csv_name_train = "bluediagonal_train.csv"
csv_name_valid = "bluediagonal_valid.csv"
sample_img_path = os.path.join(db_root_path, "bluediagonal_train")

create_dataset_based_on_rule(db_root_path, csv_name_train, csv_name_valid, valid_size=0.2, dataset_pos_samples_nb=10000,
                             dataset_neg_samples_nb=10000, rule_fun=blue_diagonal_rule)

100%|██████████| 1000000/1000000 [00:01<00:00, 971890.67it/s]


In [6]:
from gendataset import extract_sample_from_dataset
extract_sample_from_dataset(db_root_path, csv_name_train, sample_img_path, 1000, 1000)

16001it [00:00, 253035.30it/s]


In [4]:
def four_squares_total(img_content):
    sq_total = 0
    for c in img_content:
        if c["shape"] == "square":
            sq_total += 1
    return sq_total == 4

In [5]:
from gendataset import create_dataset_based_on_rule
import os
csv_name_train = "foursquares_train.csv"
csv_name_valid = "foursquares_valid.csv"
sample_img_path = os.path.join(db_root_path, "foursquares_train")

create_dataset_based_on_rule(db_root_path, csv_name_train, csv_name_valid, valid_size=0.2, dataset_pos_samples_nb=10000,
                             dataset_neg_samples_nb=10000, rule_fun=four_squares_total)

100%|██████████| 1000000/1000000 [00:05<00:00, 168775.14it/s]


In [6]:
from gendataset import extract_sample_from_dataset
extract_sample_from_dataset(db_root_path, csv_name_train, sample_img_path, 1000, 1000)

16001it [00:00, 16030.99it/s]
